In [ ]:
import requests
from bs4 import BeautifulSoup
import fitz  # PyMuPDF
import json
import time
import os
import datetime
from urllib.parse import urljoin
from tqdm import tqdm

# --- 설정 (Configuration) ---
BASE_URL = "https://www.federalreserve.gov"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
# 2000년부터 2019년까지의 URL 패턴
HISTORICAL_URL_PATTERN = "https://www.federalreserve.gov/monetarypolicy/fomchistorical{year}.htm"
# 2020년부터 현재까지의 URL
RECENT_URL = "https://www.federalreserve.gov/monetarypolicy/fomccalendars.htm"

# 사용자의 'Downloads' 폴더 경로를 동적으로 찾습니다.
# os.path.expanduser('~')는 홈 디렉터리 (예: C:\Users\YourName or /Users/YourName)를 반환합니다.
DOWNLOADS_DIR = os.path.join(os.path.expanduser('~'), 'Downloads', 'statements') # 'statements' 폴더 경로 추가
OUTPUT_FILE = os.path.join(DOWNLOADS_DIR, "fomc_statements_2000-present.jsonl")
START_YEAR = 2000
END_YEAR = datetime.datetime.now().year

# --- 헬퍼 함수 (Helper Functions) ---

def get_soup(url):
    """지정된 URL의 BeautifulSoup 객체를 반환합니다."""
    try:
        response = requests.get(url, headers=HEADERS)
        response.raise_for_status()  # HTTP 오류가 있으면 예외 발생
        return BeautifulSoup(response.text, 'html.parser')
    except requests.exceptions.RequestException as e:
        print(f"  [오류] URL에 접근할 수 없습니다: {url} ({e})")
        return None

def extract_text_from_link(link_url):
    """HTML 또는 PDF 링크에서 텍스트를 추출합니다."""
    text = ""
    try:
        # 링크가 상대 경로일 수 있으므로 절대 경로로 변환
        full_url = urljoin(BASE_URL, link_url)
        
        response = requests.get(full_url, headers=HEADERS, timeout=10)
        response.raise_for_status()

        if ".pdf" in full_url.lower():
            # PDF 처리
            with fitz.open(stream=response.content, filetype="pdf") as doc:
                for page in doc:
                    text += page.get_text() + "\n"
        else:
            # HTML 처리
            soup = BeautifulSoup(response.text, 'html.parser')
            # Fed 보도자료의 메인 콘텐츠 영역을 타겟팅
            content_div = soup.find('div', class_='col-xs-12 col-sm-8 col-md-8')
            if content_div:
                text = content_div.get_text(separator="\n", strip=True)
            else:
                # 대체 타겟팅 (구조가 다를 경우)
                text = soup.get_text(separator="\n", strip=True)
                print(f"  [경고] HTML에서 기본 콘텐츠 영역을 찾지 못했습니다. 전체 텍스트를 추출합니다. URL: {full_url}")

    except requests.exceptions.RequestException as e:
        print(f"  [오류] 링크에서 텍스트 추출 실패: {full_url} ({e})")
    except Exception as e:
        print(f"  [오류] 파일 처리 중 알 수 없는 오류: {full_url} ({e})")
        
    time.sleep(0.5)  # 서버에 부담을 주지 않기 위해 잠시 대기
    return text

def save_to_jsonl(data):
    """데이터를 JSON Lines (.jsonl) 파일에 추가합니다."""
    try:
        # 파일 저장 전 폴더가 있는지 확인하고 없으면 생성
        output_dir = os.path.dirname(OUTPUT_FILE)
        if output_dir and not os.path.exists(output_dir):
            os.makedirs(output_dir)
            print(f"  [정보] '{output_dir}' 폴더를 생성했습니다.")
            
        with open(OUTPUT_FILE, 'a', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False)
            f.write('\n')
    except IOError as e:
        print(f"[오류] 파일 쓰기 실패: {e}")

# --- 메인 크롤링 함수 ---

def crawl_historical_years(start_year, end_year):
    """2000년부터 2019년까지의 과거 데이터를 크롤링합니다."""
    print(f"--- {start_year}-{end_year} 과거 데이터 크롤링 시작 ---")
    
    for year in tqdm(range(start_year, end_year + 1), desc="연도별 진행"):
        url = HISTORICAL_URL_PATTERN.format(year=year)
        soup = get_soup(url)
        
        if not soup:
            continue
            
        # Fed 웹사이트의 과거 데이터 구조 (패널 형태)
        panels = soup.find_all('div', class_='panel')
        
        for panel in panels:
            # 날짜 정보 추출 (다양한 형식이 존재할 수 있음)
            date_element = panel.find(['div', 'h5'], class_=['panel-heading', 'fomc-meeting__date'])
            date_text = date_element.get_text(strip=True) if date_element else str(year) # 날짜를 못찾으면 연도라도 기록
            
            # 성명서(Statement) 링크 찾기
            statement_link = panel.find('a', string=lambda t: t and 'statement' in t.lower())
            
            if statement_link:
                link_url = statement_link.get('href')
                if link_url:
                    print(f"  > {year} ({date_text}) 성명서 발견. 텍스트 추출 중...")
                    text = extract_text_from_link(link_url)
                    
                    if text:
                        data = {
                            "date": date_text,
                            "year": year,
                            "source_type": "FOMC Statement",
                            "url": urljoin(BASE_URL, link_url),
                            "text": text
                        }
                        save_to_jsonl(data)

def crawl_recent_years(start_year, end_year):
    """2020년부터 현재까지의 최근 데이터를 크롤링합니다."""
    print(f"--- {start_year}-{end_year} 최근 데이터 크롤링 시작 ---")
    
    soup = get_soup(RECENT_URL)
    if not soup:
        print("[오류] 최근 데이터 페이지에 접근할 수 없습니다. 크롤링 종료.")
        return

    # 최근 데이터는 'fomc-meeting' 클래스 섹션에 연도별로 그룹화되어 있음
    year_sections = soup.find_all('div', class_='fomc-meeting--year-section')
    
    for year_section in year_sections:
        try:
            year = int(year_section.get('id'))
        except (ValueError, TypeError):
            continue
            
        if year < start_year or year > end_year:
            continue
            
        print(f"--- {year}년 데이터 처리 중 ---")
        
        meetings = year_section.find_all('div', class_='fomc-meeting')
        
        for meeting in tqdm(meetings, desc=f"{year}년 회의"):
            date_element = meeting.find('div', class_='fomc-meeting__date')
            date_text = date_element.get_text(strip=True) if date_element else str(year)
            
            # 성명서(Statement) 링크 찾기
            statement_link = meeting.find('a', string=lambda t: t and 'statement' in t.lower())
            
            if statement_link:
                link_url = statement_link.get('href')
                if link_url:
                    print(f"  > {year} ({date_text}) 성명서 발견. 텍스트 추출 중...")
                    text = extract_text_from_link(link_url)
                    
                    if text:
                        data = {
                            "date": date_text,
                            "year": year,
                            "source_type": "FOMC Statement",
                            "url": urljoin(BASE_URL, link_url),
                            "text": text
                        }
                        save_to_jsonl(data)

# --- 스크립트 실행 ---
if __name__ == "__main__":
    # 기존 파일이 있다면 삭제
    if os.path.exists(OUTPUT_FILE):
        os.remove(OUTPUT_FILE)
        print(f"기존 파일 '{OUTPUT_FILE}'을 삭제했습니다.")
        
    print("FOMC 성명서 크롤링을 시작합니다.")
    print(f"데이터 저장 위치: {OUTPUT_FILE}")
    print(f"수집 대상 기간: {START_YEAR}년 ~ {END_YEAR}년")
    
    # 2019년 말에 구조가 변경되었으므로 두 함수를 나누어 호출
    crawl_historical_years(START_YEAR, 2019)
    crawl_recent_years(2020, END_YEAR)
    
    print("--- 모든 크롤링 작업 완료 ---")
    print(f"데이터가 '{OUTPUT_FILE}' 파일에 저장되었습니다.")